In [6]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pda
import os
import seaborn as sns
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdma
import pickle
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import logging
from open_spiel.python.examples.ubc_mccfr_cpp_example import action_to_bids
from open_spiel.python.examples.ubc_nfsp_example import policy_from_checkpoint
from open_spiel.python.examples.ubc_br import BR_DIR, make_dqn_agent
from open_spiel.python.examples.ubc_utils import *
from open_spiel.python.examples.ubc_decorators import CachingAgentDecorator

from open_spiel.python.pytorch.ubc_nfsp import NFSP
import bokeh
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap

import yaml

output_notebook()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


/apps/open_spiel/venv/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['logging', 'show', 'random', 'figure', 'solve']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


Loading BokehJS ...

In [7]:
# Why is best response not at least discovering that it can get 0 reward?

In [8]:
experiment_dir = '/shared/outputs/jan2_big_game_2/lstm_deep'
checkpoint_name = 'checkpoint_3000000'
br_player = 0
report_freq = 5_000

In [9]:
checkpoint_dir = os.path.join(experiment_dir, BR_DIR)
env_and_model = policy_from_checkpoint(experiment_dir, checkpoint_suffix=checkpoint_name)
game, policy, env, trained_agents, game_config = env_and_model.game, env_and_model.nfsp_policies, env_and_model.env, env_and_model.agents, env_and_model.game_config

config_path = f'{experiment_dir}/config.yml' 

with open(config_path, 'rb') as fh:
    config = yaml.load(fh, Loader=yaml.FullLoader)
    config['num_training_episodes'] = 1_000_000
    config['rl_learning_rate'] = 0.01
    config['update_target_network_every'] = 10_000
num_training_episodes = config['num_training_episodes']

In [10]:
agents = []
for i in range(game.num_players()):
    if i == br_player:
        agents.append(make_dqn_agent(i, config, env, game, game_config))
    else:
        agent = trained_agents[i]
        agent = CachingAgentDecorator(agent)
        agents.append(agent)

logging.info(f"Training for {num_training_episodes} episodes")
# TRAINING PHASE

In [11]:
# # fill buffer with some data with opponent dropping out
# # for action in range(12):
# #     for _ in range(10):
# #         time_step_1 = env.reset()
# #         env.step([action])
# #         time_step_2 = env.step([0])

# #         agents[br_player].add_transition(
# #             time_step_1, action, time_step_2
# #         )

# for i in tqdm(range(0)):
#     time_step = env.reset()
#     while not time_step.last():
#         player_id = time_step.observations["current_player"]
#         agent = agents[player_id]
#         legal_actions = time_step.observations["legal_actions"][player_id]
#         action_list = [np.random.choice(legal_actions)]
#         time_step = env.step(action_list)
#         if player_id == br_player:
#             agent._prev_timestep = time_step
#             agent._prev_action = action

#     # Episode is over, step all agents with final info state.
#     for player_id, agent in enumerate(agents):
#         agent.step(time_step, is_evaluation=player_id != br_player)

# # # switch learning settings temporarily
# # min_buffer_size = agents[br_player]._min_buffer_size_to_learn
# # batch_size = agents[br_player]._batch_size
# # agents[br_player]._min_buffer_size_to_learn = 1
# # agents[br_player]._batch_size = 12

# # train a bunch
# for iteration in tqdm(range(1000)):
#     agents[br_player].learn()
    
# # # change settings back
# # agents[br_player]._min_buffer_size_to_learn = min_buffer_size
# # agents[br_player]._batch_size = batch_size

# # clear buffer
# agents[br_player]._replay_buffer.clear()
# agents[br_player]._target_q_network.load_state_dict(agents[br_player]._q_network.state_dict())

In [12]:
def check_on_q_values(agent, game, state=None, infostate_tensor=None, legal_actions=None, time_step=None):
    q_network = agent._q_network

    if time_step is not None:
        legal_actions = time_step.observations["legal_actions"][agent.player_id]
        it = time_step.observations["info_state"][agent.player_id]
    elif infostate_tensor is not None:
        legal_actions = legal_actions
        it = infostate_tensor
    else:
        # Extract from state
        if state is None:
            # TODO: assuming player_id on agent is 0 here, could be smarter
            state = get_first_actionable_state(game)
        legal_actions = state.legal_actions()
        it = state.information_state_tensor()

    info_state = q_network.prep_batch([q_network.reshape_infostate(it)])
    q_values = q_network(info_state).detach()[0]
    legal_q_values = q_values[legal_actions]
    legal_q_values = [agent.unmapRange(v) for v in legal_q_values]
    action_dict = get_actions(game)
    return {s: q for s, q in zip(action_dict.values(), legal_q_values)}


In [13]:
for i in range(10):
    time_step = env.reset()

    while not time_step.last():
        player_id = time_step.observations["current_player"]
        agent = agents[player_id]
        agent_output = agent.step(time_step, is_evaluation=True)
        action_list = [agent_output.action]
        if (player_id == 0):
            print(check_on_q_values(agents[0], game, time_step=time_step))

        time_step = env.step(action_list)

    episode_lengths.append(episode_length)
    # Episode is over, step all agents with final info state.
    for player_id, agent in enumerate(agents):
        agent.step(time_step, is_evaluation=True)
    print("EPISODE END")
    print()


{'Bid for 0,0,0 licenses @ $0 with activity 0': tensor(-84.4149), 'Bid for 0,0,1 licenses @ $25 with activity 25': tensor(-110.6670), 'Bid for 0,1,0 licenses @ $50 with activity 50': tensor(-72.3384), 'Bid for 0,1,1 licenses @ $75 with activity 75': tensor(-68.6537), 'Bid for 1,0,0 licenses @ $75 with activity 75': tensor(-119.6504), 'Bid for 1,0,1 licenses @ $100 with activity 100': tensor(56.4833), 'Bid for 1,1,0 licenses @ $125 with activity 125': tensor(-88.1369), 'Bid for 1,1,1 licenses @ $150 with activity 150': tensor(-188.0662), 'Bid for 2,0,0 licenses @ $150 with activity 150': tensor(-46.8630), 'Bid for 2,0,1 licenses @ $175 with activity 175': tensor(-169.8601), 'Bid for 2,1,0 licenses @ $200 with activity 200': tensor(-118.9386), 'Bid for 2,1,1 licenses @ $225 with activity 225': tensor(-151.6694)}
{'Bid for 0,0,0 licenses @ $0 with activity 0': tensor(-83.9685), 'Bid for 0,0,1 licenses @ $25 with activity 25': tensor(-113.3720), 'Bid for 0,1,0 licenses @ $50 with activity 

NameError: name 'episode_lengths' is not defined

In [ ]:
with open('output.txt', 'w') as f:
    f.write(cap.stdout)

In [ ]:
agents[br_player].loss

In [ ]:
agent = agents[br_player]
state = game.new_initial_state().child(0).child(0) \ # Chance nodes
#             .child(4).child(11) #P0 bids on 1C
#.child(1).child(11)
check_on_q_values(agent, game, state=state)

In [ ]:
import torch

agent = agents[0]
data = agent._replay_buffer._data

transitions = agent._replay_buffer.sample(agent._batch_size)

info_states = agent._q_network.prep_batch([t.info_state for t in transitions])
next_info_states = agent._q_network.prep_batch([t.next_info_state for t in transitions])

actions = torch.LongTensor([t.action for t in transitions])
rewards = torch.Tensor([t.reward for t in transitions])
iters = torch.LongTensor([t.iteration for t in transitions])

are_final_steps = torch.Tensor([t.is_final_step for t in transitions])
legal_actions_mask = torch.Tensor(np.array([t.legal_actions_mask for t in transitions]))

agent._q_values = agent._q_network(info_states)
agent._target_q_values = agent._target_q_network(next_info_states).detach()

illegal_actions = 1 - legal_actions_mask
illegal_logits = illegal_actions * -1e-9  
max_next_q = torch.max(agent._target_q_values + illegal_logits, dim=1)[0]
target = (
    rewards + (1 - are_final_steps) * agent._discount_factor * max_next_q)
action_indices = torch.stack([
    torch.arange(agent._q_values.shape[0], dtype=torch.long), actions
], dim=0)

print(target)

# [d for d in data if d.reward == agent.mapRange(0)]
# [d for d in data if d.raw_reward == 0.]

In [ ]:
import pickle
with open("stupid.pkl", 'wb') as f:
    pickle.dump(agents[br_player]._q_network.state_dict(), f)

In [ ]:
agent.mapRange(4477.8413)

In [ ]:
agents[0]._replay_buffer._data[:10]

In [ ]:
pd.Series(episode_lengths[-10000:]).describe()

In [ ]:
episode_lengths = []
for i in range(num_training_episodes):
    if i % report_freq == 0:
        print(f"----Episode {i} ---")
        print(pd.Series(episode_lengths).describe())
        logging.info(f"----Episode {i} ---")
        if i > 10_000:
            loss = agents[br_player].loss
            logging.info(f"[P{br_player}] Loss: {loss}")
            print(check_on_q_values(agents[br_player], game))

    time_step = env.reset()
    episode_length = 0

    while not time_step.last():
        episode_length += 1
        player_id = time_step.observations["current_player"]
        agent = agents[player_id]
        agent_output = agent.step(time_step, is_evaluation=player_id != br_player)
        action_list = [agent_output.action]
        time_step = env.step(action_list)

    episode_lengths.append(episode_length)
    # Episode is over, step all agents with final info state.
    for player_id, agent in enumerate(agents):
        agent.step(time_step, is_evaluation=player_id != br_player)



In [2]:
from open_spiel.python.examples.ubc_utils import *
print("HI")
smart_load_sequential_game('clock_auction', game_parameters=dict(filename='big_game_2.json', handcrafted=True)).new_initial_state().child(0).child(0).child(11).child(11).information_state_tensor()

HI


[1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 -0.25,
 19.5,
 19.25,
 29.25,
 29.0,
 48.75,
 48.5,
 58.5,
 58.25,
 78.0,
 77.75,
 2.0,
 90.75,
 60.5,
 30.25,
 2.0,
 1.0,
 1.0,
 4.0,
 2.0,
 2.0]